In [7]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp

from scipy.optimize import differential_evolution
from scipy.sparse.linalg import eigsh

# General imports
import os
import json
import numpy as np
from datetime import datetime

from qiskit.quantum_info import SparsePauliOp

# custom module
from susy_qm import calculate_wz_hamiltonian

In [27]:
# Parameters
N = 2
a = 1.0
c = 0
potential = "linear"#'quadratic'
boundary_condition = 'dirichlet'
#boundary_condition = 'periodic'

In [28]:
cut_offs_list = [8]
shots = 1024

starttime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

if shots != None:
    folder = 'Noise\\N'+str(N)
else:
    folder = 'No Noise\\N'+str(N)

#Create directory for files
base_path = r"C:\Users\Johnk\OneDrive\Desktop\PhD 2024\Quantum Computing Code\Quantum-Computing\SUSY\Wess-Zumino\VQE\DE\Files\{}\\{}\\".format(boundary_condition, folder)
os.makedirs(base_path, exist_ok=True)

print(f"Running for {boundary_condition} boundary conditions")
print(f"Running for {potential} potential")

for cut_off in cut_offs_list:

    print(f"Running for cutoff: {cut_off}")

    #calculate Hamiltonian and expected eigenvalues
    H = calculate_wz_hamiltonian(cut_off, N, a, potential, boundary_condition, c)
    
    num_eigenvalues = 6
    eigenvalues = np.sort(np.linalg.eig(H)[0][:12])
    min_eigenvalue = min(eigenvalues.real)

    #create qiskit Hamiltonian Pauli string
    hamiltonian = SparsePauliOp.from_operator(H)
    num_qubits = hamiltonian.num_qubits
    print("Num qubits: ", num_qubits)
    
    # Device
    shots = shots
    #dev = qml.device('default.qubit', wires=num_qubits, shots=shots)
    dev = qml.device('lightning.qubit', wires=num_qubits, shots=shots)


    #Initial params shape
    num_layers = 1
    params_shape = qml.StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=num_qubits)


    # Define the cost function
    @qml.qnode(dev)
    def cost_function(params):
        params = pnp.tensor(params.reshape(params_shape), requires_grad=True)
        qml.StronglyEntanglingLayers(weights=params, wires=range(num_qubits), imprimitive=qml.CZ)
        return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))
    
    
    # VQE
    vqe_start = datetime.now()

    #variables
    num_vqe_runs = 5
    max_iterations = 200
    strategy = 'best1bin'
    popsize = 15
    tolerance = 1e-2

    #data arrays
    energies = []
    x_values = []
    success = []
    run_times = []
    num_iters = []
    num_evaluations = []

    #Optimizer
    bounds = [(0, 2 * np.pi) for _ in range(np.prod(params_shape))]

    for i in range(num_vqe_runs):

        run_start = datetime.now()

        if i % 1 == 0:
            print(f"Run: {i}")

        
        # Differential Evolution optimization
        res = differential_evolution(cost_function, 
                                        bounds, 
                                        maxiter=max_iterations, 
                                        atol=tolerance,
                                        strategy=strategy, 
                                        popsize=popsize)
        
        if res.success == False:
            print("Not converged")

        energies.append(res.fun)
        x_values.append(res.x)
        success.append(res.success)
        num_iters.append(res.nit)
        num_evaluations.append(res.nfev)

        run_end = datetime.now()
        run_time = run_end - run_start
        run_times.append(run_time)

    vqe_end = datetime.now()
    vqe_time = vqe_end - vqe_start

Running for linear potential
Running for cutoff: 8
Num qubits:  8
Run: 0
Not converged
Run: 1


KeyboardInterrupt: 

In [29]:
energies

[0.0720607814582871]

In [20]:
boundary_condition

'dirichlet'

In [30]:
np.sort(eigenvalues.real)

array([4.54070242e-10, 1.11803311e+00, 1.11803311e+00, 1.11803399e+00,
       1.11803399e+00, 1.23958963e+01, 1.36092751e+01, 1.47273091e+01,
       1.47273091e+01, 1.47721877e+01, 1.47721877e+01, 1.58453431e+01])

In [8]:
#Save run
run = {
    'starttime': starttime,
    'potential': potential,
    'boundary_condition': boundary_condition,
    'c': c,
    'cutoff': cut_off,
    'num_sites': N,
    'exact_eigenvalues': [round(x.real,10).tolist() for x in eigenvalues],
    'ansatz': 'StronglyEntanglingLayers-1layer',
    'num_VQE': num_vqe_runs,
    'shots': shots,
    'Optimizer': {'name': 'differential_evolution',
                'bounds':'[(0, 2 * np.pi) for _ in range(np.prod(params_shape))]',
                'maxiter':max_iterations,
                'tolerance': tolerance,
                'strategy': strategy,
                'popsize': popsize
                },
    'results': energies,
    'params': [x.tolist() for x in x_values],
    'num_iters': num_iters,
    'num_evaluations': num_evaluations,
    'success': np.array(success, dtype=bool).tolist(),
    'run_times': [str(x) for x in run_times],
    'total_run_time': str(vqe_time)
}

# Save the variable to a JSON file
path = base_path + "{}_{}.json".format(potential, cut_off)
with open(path, 'w') as json_file:
    json.dump(run, json_file, indent=4)

In [ ]:


#base_path = r"C:\Users\Johnk\OneDrive\Desktop\PhD 2024\Quantum Computing Code\Quantum-Computing\SUSY\PennyLane\SUSY VQE\Shot Noise\Files\{}\\{}\\"
#create_vqe_plots(potential=potential, base_path=base_path, folder=folder, cut_off_list=cut_offs_list)